# Monte Carlo simulations for Urban Cooling

**Preparation of the data for the base scenario**

In [4]:
import arcpy
import pandas as pd
from arcpy.sa import *
arcpy.env.overwriteOutput = True

In [5]:
# Spatial Analyst availability
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

In [6]:
biophysical_table = pd.read_csv(
    r'C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\04_Urban_Cooling\0401_UC_Input_Files\BiophysicalTable.csv', sep =';')
print(biophysical_table)

   lucode     Kc  green_area  shade  albedo
0       1  0.200           0      0    0.17
1       3  0.001           0      0    0.14
2       6  0.800           1      0    0.16
3       7  1.000           1      1    0.20
4       9  0.001           0      0    0.14
5      15  0.001           0      0    0.20
6      16  0.001           0      0    0.25


In [11]:
## Now we want to simplify this biophysical table by combining bare land(1), Other paved (3), 
## and Paved road (9) into one entity called 'Imp' for impervious surfaces
landuse_clip = Raster(r"C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\GIS_Input_Files\LorenGIS.gdb\landuse_clip")


# Define your mapping: original -> new class code
remap = RemapValue([
    (1, 1),   # IMP - Impervious surfaces
    (3, 1),   # IMP
    (9, 1),   # IMP
    (15, 1),  # IMP
    (6, 2),   # SVEG - Shallow vegetation
    (7, 3),   # DVEG - Deep vegetation
    (16, 4)   # BLDG - Building
])

# Apply reclassification
reclass_raster = Reclassify("landuse_clip", "Value", remap)

# Optional: Save it
reclass_raster.save(r"C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\GIS_Input_Files\LorenGIS.gdb\landuse_reclass")

In [12]:
new_biophysical_table = {
    'lu_name': ['IMP','SVEG','DVEG','BLDG'],
    'Kc':     [0.001, 0.8, 1, 0.001],
    'green_area': [0, 1, 1, 0],
    'shade':      [0, 0, 1, 0],
    'albedo':     [0.14, 0.16, 0.2, 0.25]
}
df_bio = pd.DataFrame(new_biophysical_table)
print(df_bio)

  lu_name     Kc  green_area  shade  albedo
0     IMP  0.001           0      0    0.14
1    SVEG  0.800           1      0    0.16
2    DVEG  1.000           1      1    0.20
3    BLDG  0.001           0      0    0.25


In [13]:
# Create new rows for the LIDs
new_rows = pd.DataFrame([
    {
        'lu_name': 'GR5',
        'Kc': 0.5,         # assumed value for sedum
        'green_area': 1,
        'shade': 0,
        'albedo': 0.16
    },
    {
        'lu_name': 'GR20',
        'Kc': 0.6,         #assumed a bit higher 
        'green_area': 1,
        'shade': 0,
        'albedo': 0.16
    },
    {
        'lu_name': 'BC',
        'Kc': 0.9,         
        'green_area': 1,
        'shade': 0,
        'albedo': 0.16
    },
    {
        'lu_name': 'GS',
        'Kc': 0.8,          
        'green_area': 1,
        'shade': 0,
        'albedo': 0.16
    },
    {
        'lu_name': 'TRE',
        'Kc': 0.9,          
        'green_area': 1,
        'shade': 1,
        'albedo': 0.2
    }
])

# Add to original DataFrame
df_bio = pd.concat([df_bio, new_rows], ignore_index=True)
df_bio.to_csv(r'C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\04_Urban_Cooling\0401_UC_Input_Files\biophysicaltable_lid_added.csv',sep=';')
# Check result
print(df_bio)

  lu_name     Kc  green_area  shade  albedo
0     IMP  0.001           0      0    0.14
1    SVEG  0.800           1      0    0.16
2    DVEG  1.000           1      1    0.20
3    BLDG  0.001           0      0    0.25
4     GR5  0.500           1      0    0.16
5    GR20  0.600           1      0    0.16
6      BC  0.900           1      0    0.16
7      GS  0.800           1      0    0.16
8     TRE  0.900           1      1    0.20


In [16]:
## Determining the area of each landuse in each subcatchment
subcatchments = r"C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\GIS_Input_Files\Shapefiles\subcatchments_bdry.shp"
lu_reclass = r"C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\GIS_Input_Files\LorenGIS.gdb\landuse_reclass"
out_table = r"C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\GIS_Input_Files\Shapefiles\tabulate_subctmt_lu_area.dbf"
TabulateArea(
    in_zone_data=subcatchments,
    zone_field='SID',
    in_class_data=lu_reclass,
    class_field='Value',
    out_table=out_table,
    processing_cell_size=lu_reclass  # match cell size and alignment
),

(<geoprocessing server result object object at 0x000001A8659E7990>,)

In [17]:
## Since there were some discrepancies in areas in between different landuse due to pixel misalignment
## they had to be manually edited. For example some parts of roof pixels were on the pixels in a terrain subcatchment.
## These were manualy edited and put in a new table, and input into the code. 

new_table = r"C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\GIS_Input_Files\Shapefiles\tabulate_subctmt_lu_area_manualedits.dbf"
from dbfread import DBF
import pandas as pd

# Read DBF into a DataFrame
table = DBF(new_table)
lu_sub_area = pd.DataFrame(iter(table))
print(lu_sub_area),

    SID  VALUE_1  VALUE_2  VALUE_3  VALUE_4
0    S1      0.0      0.0      0.0    938.0
1    S2      0.0      0.0      0.0    494.0
2    S3      0.0      0.0      0.0    505.0
3    S4      0.0      0.0      0.0    921.0
4    S5    259.0     62.0     19.0      0.0
5    S6    118.0    770.0    227.0      0.0
6    S7    261.0     17.0      0.0      0.0
7    S8    375.0     14.0     20.0      0.0
8    S9    395.0     16.0     11.0      0.0
9   S10    376.0     68.0      0.0      0.0
10  S11    371.0      9.0     12.0      0.0


(None,)

In [18]:
## There are now two importable tables necessary for calculating Cooling capacity index (CC)
## Let's bring them into the code again.

print(lu_sub_area)     #consists area of landuse per subcatchment
print(df_bio)    # Biophysical table



    SID  VALUE_1  VALUE_2  VALUE_3  VALUE_4
0    S1      0.0      0.0      0.0    938.0
1    S2      0.0      0.0      0.0    494.0
2    S3      0.0      0.0      0.0    505.0
3    S4      0.0      0.0      0.0    921.0
4    S5    259.0     62.0     19.0      0.0
5    S6    118.0    770.0    227.0      0.0
6    S7    261.0     17.0      0.0      0.0
7    S8    375.0     14.0     20.0      0.0
8    S9    395.0     16.0     11.0      0.0
9   S10    376.0     68.0      0.0      0.0
10  S11    371.0      9.0     12.0      0.0
  lu_name     Kc  green_area  shade  albedo
0     IMP  0.001           0      0    0.14
1    SVEG  0.800           1      0    0.16
2    DVEG  1.000           1      1    0.20
3    BLDG  0.001           0      0    0.25
4     GR5  0.500           1      0    0.16
5    GR20  0.600           1      0    0.16
6      BC  0.900           1      0    0.16
7      GS  0.800           1      0    0.16
8     TRE  0.900           1      1    0.20


In [19]:
## Reshaping lu_sub_area into a long table with separate column for area
lu_long = pd.melt(
    lu_sub_area,
    id_vars='SID',
    value_vars= ['VALUE_1', 'VALUE_2', 'VALUE_3', 'VALUE_4'],
    var_name='Lu_code',
    value_name='Area'
)
lu_long['Lu_code'] = lu_long['Lu_code'].str.replace('VALUE_', '').astype(int)
print(lu_long.head())

  SID  Lu_code   Area
0  S1        1    0.0
1  S2        1    0.0
2  S3        1    0.0
3  S4        1    0.0
4  S5        1  259.0


In [20]:
## Change the codes in the table lu_long to text classes
# Example mapping from Lu_code to lu_name
lucode_map = {
    1: 'IMP',
    2: 'SVEG',
    3: 'DVEG',
    4: 'BLDG'
}

# Apply mapping
lu_long['lu_name'] = lu_long['Lu_code'].map(lucode_map)
print(lu_long.head()),

  SID  Lu_code   Area lu_name
0  S1        1    0.0     IMP
1  S2        1    0.0     IMP
2  S3        1    0.0     IMP
3  S4        1    0.0     IMP
4  S5        1  259.0     IMP


(None,)

In [21]:
## Combine subcatchment area data with biophysical table - lu_long with df_bio
df_merged = pd.merge(lu_long, df_bio, on='lu_name')
print(df_merged.head(20))

    SID  Lu_code   Area lu_name     Kc  green_area  shade  albedo
0    S1        1    0.0     IMP  0.001           0      0    0.14
1    S2        1    0.0     IMP  0.001           0      0    0.14
2    S3        1    0.0     IMP  0.001           0      0    0.14
3    S4        1    0.0     IMP  0.001           0      0    0.14
4    S5        1  259.0     IMP  0.001           0      0    0.14
5    S6        1  118.0     IMP  0.001           0      0    0.14
6    S7        1  261.0     IMP  0.001           0      0    0.14
7    S8        1  375.0     IMP  0.001           0      0    0.14
8    S9        1  395.0     IMP  0.001           0      0    0.14
9   S10        1  376.0     IMP  0.001           0      0    0.14
10  S11        1  371.0     IMP  0.001           0      0    0.14
11   S1        2    0.0    SVEG  0.800           1      0    0.16
12   S2        2    0.0    SVEG  0.800           1      0    0.16
13   S3        2    0.0    SVEG  0.800           1      0    0.16
14   S4   

In [22]:
## Create a column for cooling capacity index (CC) that uses an equation from InVEST Manual
## ETo or reference evapotranspiration is not considered here because since the whole area has
## only one value of ETo, it cancels out in the equation

df_merged['CC'] = 0.6 * df_merged['shade'] + 0.2 * df_merged['albedo'] + 0.2 * df_merged['Kc']

Ref_air_temp = 24   #Reference air temperature Tair,ref
max_temp = 25.75
uhi = max_temp -  Ref_air_temp

df_merged['T_air'] = Ref_air_temp + (1-df_merged['CC']) * uhi
print(df_merged)

    SID  Lu_code   Area lu_name  ...  shade  albedo      CC     T_air
0    S1        1    0.0     IMP  ...      0    0.14  0.0282  25.70065
1    S2        1    0.0     IMP  ...      0    0.14  0.0282  25.70065
2    S3        1    0.0     IMP  ...      0    0.14  0.0282  25.70065
3    S4        1    0.0     IMP  ...      0    0.14  0.0282  25.70065
4    S5        1  259.0     IMP  ...      0    0.14  0.0282  25.70065
5    S6        1  118.0     IMP  ...      0    0.14  0.0282  25.70065
6    S7        1  261.0     IMP  ...      0    0.14  0.0282  25.70065
7    S8        1  375.0     IMP  ...      0    0.14  0.0282  25.70065
8    S9        1  395.0     IMP  ...      0    0.14  0.0282  25.70065
9   S10        1  376.0     IMP  ...      0    0.14  0.0282  25.70065
10  S11        1  371.0     IMP  ...      0    0.14  0.0282  25.70065
11   S1        2    0.0    SVEG  ...      0    0.16  0.1920  25.41400
12   S2        2    0.0    SVEG  ...      0    0.16  0.1920  25.41400
13   S3        2    

In [23]:
df_merged.to_csv(r'C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\04_Urban_Cooling\0401_UC_Input_Files\LU_biophysicaltable_merged.csv', sep=';')

In [24]:
## Now we are interested in finding the total CC in each subcatchment regardless of the landuse
# Filter rows with non-zero Area
df_filtered = df_merged[df_merged['Area'] > 0]
df_total_temp = df_filtered.groupby('SID')['T_air'].mean().reset_index()
df_total_temp.columns = ['SID', 'Temp']
print(df_total_temp)

    SID       Temp
0    S1  25.662150
1   S10  25.557325
2   S11  25.131550
3    S2  25.662150
4    S3  25.662150
5    S4  25.662150
6    S5  25.131550
7    S6  25.131550
8    S7  25.557325
9    S8  25.131550
10   S9  25.131550


**Running for each scenario**

In [25]:
## Get the impervious and building areas for the base scenario for deducting later 
imp_base_areas = df_merged[df_merged['lu_name'] == 'IMP'][['SID', 'Area']].copy()
bdg_base_areas = df_merged[df_merged['lu_name'] == 'BLDG'][['SID', 'Area']].copy()
print(bdg_base_areas)


    SID   Area
33   S1  938.0
34   S2  494.0
35   S3  505.0
36   S4  921.0
37   S5    0.0
38   S6    0.0
39   S7    0.0
40   S8    0.0
41   S9    0.0
42  S10    0.0
43  S11    0.0


In [26]:
# Load the data
scenarios_df = pd.read_csv(r'C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\01_Preprocessing\0103_Data_cleaned_random_generated_scenarios.csv', sep = ';')
biophy_sub_df = pd.read_csv(r'C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\04_Urban_Cooling\0401_UC_Input_Files\LU_biophysicaltable_merged.csv', sep=';')
print(scenarios_df.head())
# print(biophy_sub_df.head())

biophysical_lookup = biophy_sub_df.set_index(['SID', 'lu_name'])[['Kc','shade', 'albedo', 'green_area', 'T_air']]
print(biophysical_lookup)
# Subcatchment and LID column definitions
lid_type_cols = ['S5_Type', 'S7_Type', 'S8_Type', 'S9_Type', 'S10_Type', 'S11_Type']
lid_area_cols = ['S5', 'S7', 'S8', 'S9', 'S10', 'S11']
lid_subs = ['S5', 'S7', 'S8', 'S9', 'S10', 'S11']
subcatchments = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11']



   Unnamed: 0  Type          S1  ...  S11_Type        S11    S11_Aimp
0           1   GR5    0.000000  ...       TRE   0.000000  214.960412
1           2  GR20    0.000000  ...        GS  30.094458  214.960412
2           3  GR20  938.857898  ...        BC   2.149604  214.960412
3           4  GR20  938.857898  ...        BC  30.094458  214.960412
4           5   GR5  938.857898  ...       TRE  10.000000  204.960412

[5 rows x 24 columns]
                Kc  shade  albedo  green_area     T_air
SID lu_name                                            
S1  IMP      0.001      0    0.14           0  25.70065
S2  IMP      0.001      0    0.14           0  25.70065
S3  IMP      0.001      0    0.14           0  25.70065
S4  IMP      0.001      0    0.14           0  25.70065
S5  IMP      0.001      0    0.14           0  25.70065
S6  IMP      0.001      0    0.14           0  25.70065
S7  IMP      0.001      0    0.14           0  25.70065
S8  IMP      0.001      0    0.14           0  25.700

In [28]:
import pandas as pd

# Load data
scenarios_df = pd.read_csv(r'C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\01_Preprocessing\0103_Data_cleaned_random_generated_scenarios.csv', sep = ';')
biophy_sub_df = pd.read_csv(r'C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\04_Urban_Cooling\0401_UC_Input_Files\LU_biophysicaltable_merged.csv', sep=';')
df_bio = pd.read_csv(r'C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\04_Urban_Cooling\0401_UC_Input_Files\biophysicaltable_lid_added.csv', sep=';')

# Setup
bio_lookup = df_bio.set_index('lu_name')[['Kc', 'shade', 'albedo']] # biophysical lookup table
base_lu = biophy_sub_df[['SID', 'lu_name', 'Area', 'T_air', 'Kc']]
t_air_lookup = biophy_sub_df.set_index(['SID', 'lu_name'])[['T_air']]
results = pd.DataFrame(columns=sorted(base_lu['SID'].unique()))

# Constants
ref_temp, max_temp = 24, 25.75 # reference air temperature, maximum air temperature in the area
uhi = max_temp - ref_temp

# Create function for computing the temperature after air mixing
def compute_tair(lu): 
    cc = 0.6 * lu['shade'] + 0.2 * lu['albedo'] + 0.2 * lu['Kc']  # cooling capacity
    return ref_temp + (1 - cc) * uhi

# Create a function for adding the area , Kc and Tair of certain landuse to a dataframe
def add_landuse(df, sid, lu_name, area):
    props = bio_lookup.loc[lu_name]
    t_air = compute_tair(props)
    return pd.concat([df, pd.DataFrame([{'SID': sid, 'lu_name': lu_name, 'Area': area, 'Kc': props['Kc'], 'T_air': t_air}])])

print(bio_lookup.head())
print(base_lu.head())
print(t_air_lookup.head())
print(results)

            Kc  shade  albedo
lu_name                      
IMP      0.001      0    0.14
SVEG     0.800      0    0.16
DVEG     1.000      1    0.20
BLDG     0.001      0    0.25
GR5      0.500      0    0.16
  SID lu_name   Area     T_air     Kc
0  S1     IMP    0.0  25.70065  0.001
1  S2     IMP    0.0  25.70065  0.001
2  S3     IMP    0.0  25.70065  0.001
3  S4     IMP    0.0  25.70065  0.001
4  S5     IMP  259.0  25.70065  0.001
                T_air
SID lu_name          
S1  IMP      25.70065
S2  IMP      25.70065
S3  IMP      25.70065
S4  IMP      25.70065
S5  IMP      25.70065
Empty DataFrame
Columns: [S1, S10, S11, S2, S3, S4, S5, S6, S7, S8, S9]
Index: []


In [29]:
# Simulation
for idx, row in scenarios_df.iloc[:].iterrows():
    sim_df = base_lu.copy()
    gr_type = row['Type']

    # Add green roof areas into the model
    for sid in ['S1', 'S2', 'S3', 'S4']:                   # Going through roof subcatchments
        gr_area = float(row.get(sid, 0) or 0)              # Get area of green roof from scenario
        if gr_type in bio_lookup.index and gr_area > 0:    # if green roof type is in the biophysical lookup table and is greater than 0
            mask = (sim_df['SID'] == sid) & (sim_df['lu_name'] == 'BLDG') # selects rows that match the condition
            if not sim_df[mask].empty:                     # If there is at least one row with building
                area = sim_df.loc[mask, 'Area'].values[0]  # Take the area value from the building area from base scenario dataframe
                sim_df = sim_df.drop(sim_df[mask].index)   # Removes the row with building from the base scenario 
                sim_df = add_landuse(sim_df, sid, gr_type, area) # Adds a row with green roof - uses functions that includes calculation of Tair

    # Apply LIDs to S5–S11
    for sid, lid_col, area_col in zip(['S5', 'S7', 'S8', 'S9', 'S10', 'S11'],
                                      ['S5_Type', 'S7_Type', 'S8_Type', 'S9_Type', 'S10_Type', 'S11_Type'],
                                      ['S5', 'S7', 'S8', 'S9', 'S10', 'S11']):
        lid_type = row[lid_col]                            # Get the type of LID from scenario
        lid_area = float(row.get(area_col, 0) or 0)        # Get the area of LID from scenario  
        if lid_type in bio_lookup.index and lid_area > 0:   
            sim_df = add_landuse(sim_df, sid, lid_type, lid_area)  # Add the landuse in the dataframe 
            imp_mask = (sim_df['SID'] == sid) & (sim_df['lu_name'] == 'IMP') # Get the row containing impervious surface
            sim_df.loc[imp_mask, 'Area'] -= lid_area               # Deduct the LID area added from the impervious area


    # Weighted mean temperature
    weighted = (sim_df['T_air'] * sim_df['Area']).groupby(sim_df['SID']).sum() / sim_df.groupby('SID')['Area'].sum()
    weighted = weighted.round(2)
    results.loc[f"Sim{idx}"] = weighted.round(2)
    results['Mean'] = results.mean(axis=1).round(2)
    

# Show result
print(results.head())


         S1    S10    S11     S2     S3  ...     S6     S7     S8     S9   Mean
Sim0  25.66  25.66  25.65  25.66  25.66  ...  25.21  25.68  25.62  25.65  25.60
Sim1  25.66  25.65  25.63  25.66  25.66  ...  25.21  25.61  25.59  25.63  25.57
Sim2  25.48  25.65  25.65  25.66  25.66  ...  25.21  25.68  25.59  25.64  25.57
Sim3  25.48  25.63  25.63  25.48  25.66  ...  25.21  25.68  25.62  25.63  25.57
Sim4  25.52  25.56  25.62  25.66  25.66  ...  25.21  25.67  25.60  25.64  25.58

[5 rows x 12 columns]


In [31]:
results.to_csv(r'C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\04_Urban_Cooling\0403_Temp_diff_results.csv', sep =';')